In [1]:
'''
#Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

path = "comparisonTest-AngelaAttempt1.txt"
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Using TensorFlow backend.


corpus length: 6029
total chars: 30
nb sequences: 1997
Vectorization...
Build model...

Epoch 1/60
1997/1997 [==============================] - 3s 2ms/step - loss: 3.0960

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "m ever assembled history politics made h"
m ever assembled history politics made ho  e  e e e aeae  s e e     ea a en ea  oe n  n e  ne an aeee e  ea   ae  a e n e oae na     ae   n ae aaoen   aaae e  eo oea       eo   ee oa e n  e e  on n  a  e   ae  a aeaeaae  aoe  a a  e a eo  ae a ae   a aao  e eoe ea    ao e a ne ae  eae ae  eeea  a ee  e  ae a a ee eeao e e t e e ee ae   aae  aae ie o oe   ae a aa  ae e  ne oa  n a e an ae  a e  re a aa e ee ee ne aaa an a   e  o e  e   a
----- diversity: 0.5
----- Generating with seed: "m ever assembled history politics made h"
m ever assembled history politics made h nes ei ne noropeeaee o o  ceoo  an oii ea ea vh  esoonnea  nso o eope s  yaa eoeooileun cr  g  e ae  n  ioeteu aoe  oaeot   i

ss tonight know debt beyond measure campe seu aee  are ele sen sure awe bee  der tee re tee ne ee ler  are  el d rere tere lere bee caree ers her ene nere ber eeruce s ue ce  ee ewee beree eee ee cere cere ee pere ten eune uren pet ae sere yer ere ger teer se ee sene ree les are te ee ten rere te ceas ee  ene ee  ter ree ger tee cune  ane ere teme  ee tetre dew ane ere see  ee  heet ter erere ate  are sere suge pere eres  are eere pere 
----- diversity: 1.0
----- Generating with seed: "ss tonight know debt beyond measure camp"
ss tonight know debt beyond measure campe eet ganve rspdeses rekee derod uee tan  le  onechey ele neregd rcere unr gur fared leso c ge shew y unite y whet tor serhie  orerepangy moct getet aes hrre cerribet dotithyes rale  ans jey ged waun eeret  roenen nte giucaesek hore tagln ateo e rey ckef pe y yt ieh clded pene lereedel cede s belh ugecpurstpe cer sed hee nlhe de pdnwet donin s cor  rewkrrongehtirug  auterls siey beke dsoren  afetn
----- diversity: 1.2
-----

ommon purpose yes bombs fell harbor tyratu mame uxand maves  allhathateditibreals wtryfit crotl trlit olingeny coupn abtiopt faols lcunginssyen bardarn mounryucialeliriysssiveellot aastotinstionehlingstyol hime hotlroid aset ppllkses t uning tiome nshuce licany wollestalt prafe net imp ysy0on sumond wiujime wenccgwoaigont amh matwert omeopbats ssaticatratrecdrlrtwret wornimaltgstharsaiteimeamertmesthlomiuseios lwobemarston ther carentin
Epoch 8/60
1997/1997 [==============================] - 3s 1ms/step - loss: 2.1566

----- Generating text after Epoch: 7
----- diversity: 0.2
----- Generating with seed: " dream reaffirm fundamental truth many o"
 dream reaffirm fundamental truth many one songe ton gon hon sone mon  angen con ere tone songen cong non ton hone ton hon en conge tong ton  one pone son son gon gon sone sone ton gon en wonge  one pone pone song nen ton en con gon gon gon song ton gon song ton son hong ton hon get ann wonge ton sone tone mon song tone tone pone congen conge t

 sixteen years rock family love life nation toun wamar mad stion surew des fatwert loungat ghleat fealion wall aved serect halicam atwoung cale furrican grew ratrery anew dert lime thamanght tropuer falworce toricans halion unowore thamuny halw rathery halwery yewick arew came toon hun hamica k woll hatn wart anwword beice sewarce proald ghtimon chatach prote warw res coll suntworl caew fone chen suwarnght hallounght warihad halworld gh
----- diversity: 1.0
----- Generating with seed: " sixteen years rock family love life nat"
 sixteen years rock family love life nataven hawion answ louct uniopk chappons caltors descaan tolonge litiugherliry arefubest pollicightthalsharewtd onercvetight mekionigwt veclerdalstaoliny uawtupa unhwar ges heeivente dise meren nonion cuntwoen salwed bresd seele belack jud ayoort anilon sew malw deard worsderyer fallooved fimorqtath toreamucht spoey hais cowiiged chman stours leved sew calpary ved proflerss alewones hatworghamary hi
----- diversity: 1.2
-----

...

e years block block brick brick callouse cantion americans lwonges new erect finewr camerica leament telilent one peopee peoped sile mone chews fiss sericanshe halloves fall americant onew men cesplon grew streek forch people bivis siak stive ser congry campary annind parnica serec mest thonger canngc camec wass preck wampnitidation vid lionger camen geoner canngs callessed wame heprongwon bringimes malleved geongen carl ament toone peo
----- diversity: 1.0
----- Generating with seed: "e years block block brick brick callouse"
e years block block brick brick callouse cant leopeb handericans wall waspang one peopee people beck woment tive touss grew sustrongs nativod brendes preaces alw campresp breck slivess darid ged gren americant t ling ged sunetred greaces mack way fonger cames anwore proved get unerect faricid dawn tiun volle sed hope peoppe parivisto beridary twonger congucy unerect serdebed brevery amer cant leoreb monges warthatng maches brmeen tames 
----- diversity: 1.2
-----

ike millions others stood line make voice harst unew atrong falives  reace hope peopperica chieves prived ser world sewd tomes preared thongery vile ser wongerican geeared hearsermat despaght laty ssitid get letcamerican eled seldersed hed blen s armort anninin nourqed yifited stimed geserstlonge inatid despion dass conliust anew ats natco norich prouidery arld sharifean peace hrery biting toople pphopperica serechice americant unehuder
Epoch 50/60
1997/1997 [==============================] - 2s 1ms/step - loss: 0.0276

----- Generating text after Epoch: 49
----- diversity: 0.2
----- Generating with seed: "oss land saw nation conquer fear new dea"
oss land saw nation conquer fear new deam ve wanlery hien despars ploved grew streel userbld stion grew streed conpich parlicaragrect uneames chalerss wall susercambentig teople peoppe peoped timelluss hamed brermer get serbert one brend shourtion protid stroell tion erection ericans wall happont onited timellons ghtals purftrengs alw rmess g

lf reliance individual liberty national callensed timel userved toonger cann w chiegs camp brendes proved siwarse serve sericansew hell brendes proved stied sulad ener stime ser spion come hald shied gener hamer campary america as areracan hallers alw reaches brecd ry american greales lise new stica sear hoves campary america as arericans warin toung hed liene  ass want aninid sered strengs  leaker century trmery hale serer longe  fall 
----- diversity: 1.0
----- Generating with seed: "lf reliance individual liberty national "
lf reliance individual liberty national disid dest timellussed hied new despict aniinin warigh native americanth toll  ved stive sep brendist tall wust nerice refper camnghang athived seled streed shourt hel brerpt s iongak stion mouched tionger campivid tooke peoppe parld masn tiviged troel hass strange  vid strames cartory blendes wary americantew hel  unearnes falld seserbect toonger camp brinder longe hive red sestre les ass cambrit
----- diversity: 1.2
-----

ldren fall asleep wonder make mortgage proeld streek respad stined susk tonk ratren ss americant one prespse shaveris avived sed strted ges lon brentid onerica sew averica agricars alworsertcy ushared yes sullaced trouge nos stcampation shation vo con vid gren sed strallen joonged ghted sust neppbrst cannight atworless gwowlent ditionl unewes ffallent toongh people chiearst anmor spive stated timelwuss lived serffed suwadt teopge tion w
Epoch 57/60
1997/1997 [==============================] - 3s 1ms/step - loss: 0.0676

----- Generating text after Epoch: 56
----- diversity: 0.2
----- Generating with seed: "ury daughters lucky live long ann nixon "
ury daughters lucky live long ann nixon coppe chinge conn chee blect sarataon carl amen comerica rew aspang amoriched thought lition alwt longer cann s aricanca les brerds alw des cant seation axotin tion prifid stoted grew sered streed greanes alwace haplon one prespe brecdess thoplery datid des alwonse longes campary america charvid get des

iend last sixteen years rock family love bespsed arrender lacled ges despacs prespies pailisers get dane respacs proved yeople mak cann crivid ge alaries fall aved shie despen wonlion brick wary amen comp brend s world sericanco unimery tronger campgec cant uneaprenchs atrecd senicing touight keope peoped proced arworsers darlon fiss als racan conig tes muchee husserled arstren sereames preced americant one prespied goted brender looker
----- diversity: 1.0
----- Generating with seed: "iend last sixteen years rock family love"
iend last sixteen years rock family love changh aninct ves brick alin toople phoved streek red hameblition derifast andimen get vellonserthople yeonge tronger cant speamuna len ges asking mary america as arshanig tatight voten let beskers americans warion conqer cericar grew amenica beach suwart unewre hepebrention alw ifed seriscion palivid des affecd respion pricid stronge  fellendes proved seracarrew brecker thiek hepen wans plotid s
----- diversity: 1.2
-----